In [0]:
import tweepy #https://github.com/tweepy/tweepy
import csv
import json
import re
import time

with open('cred.json') as json_file:
    data = json.load(json_file)

#Twitter API credentials
consumer_key = data['consumer_key']
consumer_secret = data['consumer_secret']
access_key = data['access_key']
access_secret = data['access_secret']


In [0]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
accounts = '@naval, @Via_Benjamin, @RortyWitt, @EdLatimore, @StoopToRise, @AymPlanet, @Wealth_Theory, @alanhliang, @TradingNirvana, @modestproposal1, CrazyPolymath, @SentientBonobo, @webdevMason, @stoic_dilettant, @AJA_Cortes, @martyrmade, @PresentWitness_, @millstoic, @z3nblack, @TheChuChu_, @orangebook_, @yawyr_vk, @Noahpinion, @ThomasSowell, @LifeMathMoney, @DejaRu22, @lawsofaurelius, @48_quotes, @shl, @cryptoseneca, @paulg, @TheCreativeFury, @Kpaxs, @TheAncientSage, @DeeperThrill, @mmay3r, @DrRalphNap, @TheStoicEmperor, @uncannyinsights'
accounts = accounts.split(', ')
accounts = [x[1:] for x in accounts]
print(accounts)

['naval', 'Via_Benjamin', 'RortyWitt', 'EdLatimore', 'StoopToRise', 'AymPlanet', 'Wealth_Theory', 'alanhliang', 'TradingNirvana', 'modestproposal1', 'razyPolymath', 'SentientBonobo', 'webdevMason', 'stoic_dilettant', 'AJA_Cortes', 'martyrmade', 'PresentWitness_', 'millstoic', 'z3nblack', 'TheChuChu_', 'orangebook_', 'yawyr_vk', 'Noahpinion', 'ThomasSowell', 'LifeMathMoney', 'DejaRu22', 'lawsofaurelius', '48_quotes', 'shl', 'cryptoseneca', 'paulg', 'TheCreativeFury', 'Kpaxs', 'TheAncientSage', 'DeeperThrill', 'mmay3r', 'DrRalphNap', 'TheStoicEmperor', 'uncannyinsights']


In [0]:
 class Tweet(object):
    def __init__(self):
        self.author_name = ''
        self.handle = ''
        self.tweet_content = ''
        self.likes = -1
        self.retweets = -1
        self.created_at = None
    
    def parse_text(self, text, additional=None):
        text = text[:text.rfind('http')]
        self.tweet_content = text
        if additional is not None:
            additional = '<Q> ' + additional[:additional.rfind('http')] + ' </Q>'
            self.tweet_content = self.tweet_content + additional
    
    def __repr__(self):
        return {'author_name':self.author_name, 'handle':self.handle, 'tweet_content':self.tweet_content, 'likes':self.likes, 'retweets':self.retweets, 'created_at':self.created_at}

    def __str__(self):
        return 'author_name: ' + self.author_name + ' -- handle: ' + self.handle + ' -- tweet_content: \n' + self.tweet_content + '\n likes: ' + str(self.likes) + ' -- retweets: ' + str(self.retweets) + ' -- created_at: ' + str(self.created_at)

In [0]:
def parse_tweet(tweet_obj):
    try:
        tweet = Tweet()
        if(tweet_obj.full_text[:2] == 'RT'):
            retweeted = tweet_obj.retweeted_status
            if retweeted.user.screen_name in accounts:
                return None
            tweet = parse_tweet(retweeted)
        elif(tweet_obj.is_quote_status):
            r_tweet = api.get_status(
                tweet_obj.quoted_status_id, tweet_mode='extended')
            if(r_tweet.in_reply_to_status_id is not None):
                r_tweet.full_text = r_tweet.full_text[r_tweet.full_text.find(
                    '-') + 2:]
            tweet.author_name = tweet_obj.user.name
            tweet.handle = tweet_obj.user.screen_name
            tweet.parse_text(tweet_obj.full_text, additional=r_tweet.full_text)
            tweet.likes = tweet_obj.favorite_count
            tweet.retweets = tweet_obj.retweet_count
            tweet.created_at = tweet_obj.created_at

        else:
            tweet.author_name = tweet_obj.user.name
            tweet.handle = tweet_obj.user.screen_name
            tweet.parse_text(tweet_obj.full_text)
            tweet.likes = tweet_obj.favorite_count
            tweet.retweets = tweet_obj.retweet_count
            tweet.created_at = tweet_obj.created_at
        return tweet
    except:
        return None

In [0]:
def fetch_all_tweets(screen_name):
    all_objects = []
    # make initial request for most recent tweets (200 is the maximum allowed count)
    try:
        try:
            new_tweets = api.user_timeline(screen_name=screen_name, count=200, tweet_mode='extended')
        except tweepy.error.RateLimitError:
                    time.sleep(15*60)
                    new_tweets = api.user_timeline(screen_name=screen_name, count=200, tweet_mode='extended')
        # save the id of the oldest tweet less one
        oldest = new_tweets[-1].id - 1
        for tweet in new_tweets:
            if(tweet.in_reply_to_status_id is None):
                tweet_obj = parse_tweet(tweet)
                if tweet_obj is not None:
                    all_objects.append(tweet_obj)
            while len(new_tweets) > 0:
                try:
                    print("getting tweets before %s" % (oldest))
                    # all subsiquent requests use the max_id param to prevent duplicates
                    new_tweets = api.user_timeline(screen_name=screen_name, count=200, max_id=oldest, tweet_mode='extended')
                    # update the id of the oldest tweet less one
                    if len(new_tweets) == 0:
                        break
                    oldest = new_tweets[-1].id - 1
                    for tweet in new_tweets:
                        if(tweet.in_reply_to_status_id is None):
                            tweet_obj = parse_tweet(tweet)
                            if tweet_obj is not None:
                                all_objects.append(tweet_obj)
                    print("...%s tweets downloaded so far" % (len(all_objects)))
                except tweepy.error.RateLimitError:
                    time.sleep(15*60)
                    continue
    except tweepy.error.TweepError:
        pass
    return all_objects

In [24]:
final_list = []
for account in accounts:
    print(account)
    final_list += fetch_all_tweets(account)

naval
getting tweets before 1159244393968099327
...29 tweets downloaded so far
getting tweets before 1146177940960366591
...51 tweets downloaded so far
getting tweets before 1140065813090619391
...76 tweets downloaded so far
getting tweets before 1133138076845219840
...111 tweets downloaded so far
getting tweets before 1123765594938060800
...169 tweets downloaded so far
getting tweets before 1107112820078833663
...211 tweets downloaded so far
getting tweets before 1090840497218674689
...270 tweets downloaded so far
getting tweets before 1083509734999638015
...369 tweets downloaded so far
getting tweets before 1064100136379273215
...433 tweets downloaded so far
getting tweets before 1054516197499138047
...491 tweets downloaded so far
getting tweets before 1041002925760249861
...516 tweets downloaded so far
getting tweets before 1029943576241819647
...518 tweets downloaded so far
getting tweets before 1029265410300960767
...542 tweets downloaded so far
getting tweets before 1018342824712

In [0]:
import pandas as pd

In [0]:
df = pd.DataFrame.from_records([s.__repr__() for s in final_list])

In [32]:
print(df.shape)

(31115, 6)


In [33]:
df.describe()

,likes,retweets
count,3.111500e+04,31115.000000
mean,1.044935e+03,342.258236
std,1.221500e+04,3887.307426
min,0.000000e+00,0.000000
25%,1.400000e+01,2.000000
50%,6.900000e+01,13.000000
75%,2.850000e+02,65.000000
max,1.352225e+06,424270.000000


In [0]:
df.to_csv('/content/gdrive/My Drive/tweets.csv', index=False)

In [35]:
df.author_name.unique()

array(['Naval', 'Ammar', 'Matt Woods', ..., 'Oscar Wilde',
       'European Beauty', 'Eric Peng'], dtype=object)

In [0]:
df.to_csv('./tweets.csv', index=False)